In [ ]:
- Bình thường, khi đăng nhập mình chỉ lưu `access_token`
- Bây giờ mình cần lưu thêm cả `refresh_token` nữa

In [ ]:
//...

function Login() {
    const navigate = useNavigate();
    const { isSuccess } = useMeQuery();
    const [login, response] = useLoginMutation();

    useEffect(() => {
        if (isSuccess) navigate("/");
    }, [isSuccess, navigate]);

    useEffect(() => {
        if (response.isSuccess) {
            /* Lưu thêm refresh_token vào localStorage */
            const { access_token, refresh_token } = response.data;
            localStorage.setItem("accessToken", access_token);
            localStorage.setItem("refreshToken", refresh_token);
            navigate("/");
        }
    }, [response, navigate]);

    //...

    const handleLogin = (credentials) => {
        login(credentials);
    };

    /* JSX */
    return (
        <div>
            ...
        </div>
    );
}

export default Login;


In [ ]:
// src/utils/httpRequest

// 1. import axios
import axios from "axios";

// 2. Cấu hình instance chứa các config
const httpRequest = axios.create({
    baseURL: "https://api01.f8team.dev/api",
});

httpRequest.interceptors.request.use((config) => {
    const accessToken = localStorage.getItem("accessToken");
    if (accessToken) config.headers.Authorization = `Bearer ${accessToken}`;

    return config;
});

/* Lưu ý các instance sẽ là riêng, axios riêng, httpRequest riêng */
axios.interceptors.response.use((response) => {
    return response.data;
});

/* Bản chất của interceptors là nhận vào cái gì thì cần phải trả ra cái đó:
- Nếu không trả ra thì sẽ không trả ra cái gì ở kết quả cuối cùng
VD gửi request -> interceptors request -> server nhận request -> interceptors response -> response

+ Nếu tại các mắt xích interceptors mà không trả ra cái gì, liên kết sẽ bị phá vỡ dẫn đến kết quả cuối cùng bị undefined
*/
httpRequest.interceptors.response.use(
    (response) => {
        return response.data;
    },
    /* Xử lý khi thất bại */
    (error) => {
        /* error: Object lỗi AxiosError:
            + Có trường config chính là config của axios    
            + Đây chính là cục config bị lỗi
            + Khi refresh token thành công (bước 8) -> quay lại để gọi API với cục config bị lỗi này (gọi lại bước 5 trong luồng)
            + Khi gọi lại API thì nó lại lọt vào interceptors request để kiểm tra
        */

        const refreshToken = localStorage.getItem("refreshToken");
        /* 
        1. Kiểm tra token hết hạn với lỗi 401 + phải có refreshToken thì mới cho gọi là refreshToken.
        2. Nếu không có thì đá sang trang đăng nhập luôn

        */
        if (error.status === 401 && refreshToken) {
            /* 
                - Không nên sử dụng httpRequest.post(...)
                - Dễ bị đệ quy nếu refresh_token hết hạn
                    + refresh_token hết hạn
                    + Gọi API => lỗi 401
                    + lọt vào interceptors của chính instance `httpRequest`
                    + lại gọi API tiếp...
                => gọi bằng instance của axios để nó không bị lọt vào interceptors của instance `httpRequest`
            */

            /* 2. Gọi API refresh với baseURL từ cục config */
            const original = error.config;

            axios
                .post(`${original.baseURL}/auth/refresh-token`, {
                    refresh_token: refreshToken,
                })
                .then((response) => {
                    /* Lưu token vào localStorage:
                        - Kết quả: 
                        => Login => access_token + refresh_token
                        => access_token hết hạn 
                        => gọi refresh
                        => lấy token mới
                        => f5 trang để gọi lại API với token mới
                        => Thành công!
                    */

                    /* Tại sao ở trang Home
                    => gọi /me thất bại
                    => gọi /auth/refresh_token thành công
                    => Nhưng vẫn bị văng sang trang Login vì lỗi API 401 ở /me. Cụ thể:
                        + Khi /me bị lỗi => lọt vào khối error trong interceptors.response 
                        + gọi /auth/refresh_token là một Promise (bất đồng bộ)
                        + Không chờ Promise chạy xong, mà code chạy tiếp thẳng xuống dòng
                        return Promise.reject(error) => lỗi 401
                        + Rồi tại Private.jsx sẽ kiểm tra isError bị đánh true
                        + Đẩy sang trang Login
                    => Trong khi Promise đang xảy ra chúng ta cần một khoảng dừng lại. Chờ đến khi Promise chạy xong và trả kết quả, nếu thành công thì refresh token và gọi lại config lỗi, còn thất bại thì return Promise.reject()
                    */
                    const { access_token, refresh_token } = response.data;
                    localStorage.setItem("accessToken", access_token);
                    localStorage.setItem("refreshToken", refresh_token);
                });

            // Nếu không lưu vào localStorage thì:
            // nếu f5 thì gọi API refresh sẽ bị lỗi
            // Vì lần đầu gọi API refresh thành công
            // Trả về token mới, kill token cũ
            // nhưng trong localStorage vẫn đang lưu token cũ => lỗi

            // Để lấy ra dữ liệu cần object {access_token...} thì cần response.data.access_token
            // Vì chưa có interceptors của axios
        }

        // Trả về lỗi => lọt vào catch
        return Promise.reject(error);
    },
);

export default httpRequest;


In [ ]:
// 1. import axios
import axios from "axios";

// 2. Cấu hình instance chứa các config
const httpRequest = axios.create({
    baseURL: "https://api01.f8team.dev/api",
});

//...

httpRequest.interceptors.response.use(
    (response) => {
        return response.data;
    },
    // Xử lý async/await để không bị lọt vào dòng return Promise.reject(error) khi chưa chạy xong
    async (error) => {
        const refreshToken = localStorage.getItem("refreshToken");
        if (error.status === 401 && refreshToken) {
            const original = error.config;

            try {
                const response = await axios.post(
                    `${original.baseURL}/auth/refresh-token`,
                    {
                        refresh_token: refreshToken,
                    },
                );
                const { access_token, refresh_token } = response.data;
                localStorage.setItem("accessToken", access_token);
                localStorage.setItem("refreshToken", refresh_token);

                // Gọi lại cục config bị lỗi (Retry gọi lại)
                return await httpRequest(original);
            } catch (error) {
                // Khi refresh token thất bại => đá sang login
                return Promise.reject(error);
            }
        }
    },
);

export default httpRequest;


In [ ]:
//src/pages/Home/index.jsx

import { useDevicesQuery, useMeQuery } from "@/services/auth";
import {
    useCreateProductMutation,
    useGetProductsQuery,
} from "@/services/product";
import React from "react";

function Home() {
    // ...
    const [createProduct] = useCreateProductMutation();
    const { isSuccess, data: currentUser } = useMeQuery();

    // Gọi thêm một API bảo vệ để mô phòng case gửi một lúc nhiều request lỗi
    const { data: devices } = useDevicesQuery();
    console.log(devices);

    //...

    return (
        <div>
            {isSuccess && <h2>Xin chào {currentUser.firstName}</h2>}
            <button onClick={handleCreateProduct}>Create New Product</button>
            <h1>Product List</h1>
            {/* ... */}
        </div>
    );
}

export default Home;


In [ ]:
# Các case khi gọi gọi nhiều API 401 cùng lúc

## Case 1: `/me` và `/devices` gọi cùng lúc `access_token` hết hạn?
- `/me` và `/devices`:
  1. cùng lỗi 401
  2. cùng gọi API `/auth/refresh-token` với cùng `refresh_token`
  3. Phụ thuộc tốc độ mạng:
    * Có những lúc chúng được gửi gần như là song song
    * Có những lúc chúng được gửi xa nhau
  4. Request nào được xử lý trước => Success, xử lý sau => Thất bại (401)

## Case 2: `/me` và `/devices` gọi cùng lúc `access_token` hết hạn?
- `/me` và `/devices`:
  1. cùng lỗi 401
  2. cùng gọi API `/auth/refresh-token` với cùng `refresh_token`
  3. Lần này thời gian tranh nhau chỉ vài ms, gần như là cùng lúc:
  - Giả sử: Khi gửi Request đến Authorization Server
    + Để kiểm tra refresh_token có phải hợp lệ không, thì nó sẽ phải lấy từ trong DataBase để đối chiếu, nếu bằng nhau hoặc còn hạn thì là hợp lệ!
    + Giả định gửi 2 `refresh_token` là hợp lệ
  - Auth Server liên lạc với DataBase bằng cách gửi một httpRequest để nhận về response để kiểm tra, đối chiếu

  - VD: thời gian response trả về từ Database cho Auth Server là 50ms
  - Trong khi đó 2 `refresh_token` chỉ chênh nhau 10ms (tức là response từ DataBase vẫn chưa trả về để đối chiếu)
  - Tương tự như 2 ông đã kịp chui vào cửa trước khi kiểm tra và thay đổi "mã bí mật"
  - Kết quả: sau khi response từ DB trả về thành công, nó nhận thấy có 2 Request từ phía Client có `refresh_token` đều hợp lệ --> nên cả 2 sẽ thành công?
  - Mà cả thành công thì sẽ tạo rả 2 cặp `access_token` và `refresh_token` mới mặc dù lúc gửi đi thì chỉ mang chung `refresh_token`
  => Thực ra là ở phía CLient, thằng nào trả về sau sẽ ghi đè thằng trả về trước => nhận 1 cặp `access_token` và  `refresh_token` mới nhất

  3. Nếu request cùng thành công nếu ở bước kiểm tra cùn thấy thành công!

# Giải pháp:
- Đặt ra một quy định:
- Không cho 10 ông request lao vào cùng lúc nữa
- Yêu cầu luôn: Chỉ riêng ông đầu tiên được vào thôi
- Ông cầm "chìa , ổ mới" --> xong phát cho tất cả các ông còn lại